In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import scipy
import statsmodels as stm
import sklearn

In [7]:
from sklearn.neighbors import KDTree, BallTree, KNeighborsClassifier, RadiusNeighborsClassifier, DistanceMetric

In [ ]:
KDTree()

In [ ]:
KNeighborsClassifier()

In [ ]:
RadiusNeighborsClassifier()

### **Before we work with KNN and other distance dependent algorithms lets understand the various distance metrics. My motive in understanding the distances because there is no one metric that can be used for every problem.**

In [60]:
X = np.array([[1,2,3]])
Y = np.array([[3,4,5]])
X,Y

(array([[1, 2, 3]]), array([[3, 4, 5]]))

In [61]:
X.shape, Y.shape, X.ndim, Y.ndim

((1, 3), (1, 3), 2, 2)

In [51]:
eu_dist = DistanceMetric.get_metric('euclidean')

In [52]:
man_hat_dist = DistanceMetric.get_metric('manhattan')

In [53]:
eu_dist.pairwise(X,Y)

array([[3.46410162]])

In [54]:
man_hat_dist.pairwise(X,Y)

array([[6.]])

#### **CASE-II: Different shape vectors**

In [57]:
XX = X.copy().reshape(3,1)
YY = Y.copy().reshape(3,1)

In [62]:
XX,YY

(array([[1],
        [2],
        [3]]),
 array([[3],
        [4],
        [5]]))

In [63]:
eu_dist.pairwise(XX,YY)

array([[2., 3., 4.],
       [1., 2., 3.],
       [0., 1., 2.]])

In [64]:
man_hat_dist.pairwise(XX,YY)

array([[2., 3., 4.],
       [1., 2., 3.],
       [0., 1., 2.]])